In [1]:
from summarynb import show, indexed_csv, table, chunks
from malid.external.summarynb_extras import plaintext, empty
from malid import config, logger
import pandas as pd
from IPython.display import display, Markdown
from malid.trained_model_wrappers import RepertoireClassifier

In [2]:
fold_label_train, fold_label_validation = config.get_fold_split_labels()

# Summary statistics of the repertoire --> classifiers

## Features for each specimen's repertoire, per isotype:

Derived from same sampled versions of repertoires used for other models:

* Top 15 PCs from V-J gene use counts
* Median sequence SHM rate
* Proportion of sequences at least 1% mutated

All standardized to zero mean and unit variance.

_Some models will not have feature importances._

In [3]:
for gene_locus in config.gene_loci_used:
    # map target_obs_column to results_output_prefix
    targets = {}
    for target in config.classification_targets:
        targets[target] = (
            RepertoireClassifier._get_output_base_dir(
                gene_locus=gene_locus,
                target_obs_column=target,
                sample_weight_strategy=config.sample_weight_strategy,
            )
            / "train_smaller_model"
        )

    print(targets)

    for target_obs_column, results_output_prefix in targets.items():
        display(
            Markdown(
                f"# {gene_locus}, {target_obs_column} trained on {fold_label_train} set"
            )
        )

        display(Markdown(f"## Specimen predictions on {fold_label_validation} set"))
        try:
            ## All results in a table
            all_results = pd.read_csv(
                f"{results_output_prefix}.compare_model_scores.tsv",
                sep="\t",
                index_col=0,
            )
            show(table(all_results), headers=["All results, sorted"])

            models_of_interest = all_results.index

            ## Confusion matrices
            for model_names in chunks(models_of_interest, 4):
                show(
                    [
                        [
                            plaintext(
                                f"{results_output_prefix}.classification_report.{model_name}.txt"
                            )
                            for model_name in model_names
                        ],
                        [
                            f"{results_output_prefix}.confusion_matrix.{model_name}.png"
                            for model_name in model_names
                        ],
                        # mistakes
                        [
                            f"{results_output_prefix}.confusion_matrix.{model_name}.binary_vs_ground_truth.png"
                            for model_name in model_names
                        ],
                        # feature importances
                        [
                            f"{results_output_prefix}.feature_importances.{model_name}.png"
                            for model_name in model_names
                        ],
                    ],
                    headers=model_names,
                    max_width=500,
                )
        except FileNotFoundError as err:
            logger.warning(f"Not run: {err}")

        display(
            Markdown(
                "## Apply train-smaller model -- Test set performance - With and without tuning on validation set"
            )
        )
        try:
            ## All results in a table
            all_results = pd.read_csv(
                f"{results_output_prefix}.compare_model_scores.test_set_performance.tsv",
                sep="\t",
                index_col=0,
            )
            show(table(all_results), headers=["All results, sorted"])

            models_of_interest = all_results.index

            ## Confusion matrices
            for model_names in chunks(models_of_interest, 4):
                show(
                    [
                        [
                            plaintext(
                                f"{results_output_prefix}.test_set_performance.{model_name}.classification_report.txt"
                            )
                            for model_name in model_names
                        ],
                        [
                            f"{results_output_prefix}.test_set_performance.{model_name}.confusion_matrix.png"
                            for model_name in model_names
                        ],
                        # mistakes
                        [
                            f"{results_output_prefix}.test_set_performance.{model_name}.confusion_matrix.binary_vs_ground_truth.png"
                            for model_name in model_names
                        ],
                    ],
                    max_width=500,
                    headers=model_names,
                )
        except FileNotFoundError as err:
            logger.warning(f"Not run: {err}")

        display(Markdown("---"))

{<TargetObsColumnEnum.disease: TargetObsColumn(obs_column_name='disease', is_target_binary_for_repertoire_composition_classifier=False, available_for_cross_validation_split_strategies={<CrossValidationSplitStrategy.in_house_peak_disease_timepoints: CrossValidationSplitStrategyValue(data_sources_keep=[<DataSource.in_house: 1>], subtypes_keep=['Covid19 - Sero-positive (ICU)', 'Covid19 - Sero-positive (Admit)', 'Covid19 - Acute 2', 'Covid19 - Admit', 'Covid19 - ICU', 'Influenza vaccine 2021 - day 7'], diseases_to_keep_all_subtypes=['Healthy/Background', 'HIV', 'Lupus', 'T1D'], filter_specimens_func_by_study_name={'Covid19-buffycoat': <function acute_disease_choose_most_peak_timepoint at 0x7f366d58b9d0>, 'Covid19-Stanford': <function acute_disease_choose_most_peak_timepoint at 0x7f366d58b9d0>}, gene_loci_supported=<GeneLocus.BCR|TCR: 3>, exclude_study_names=['IBD pre-pandemic Yoni'], study_names_for_held_out_set=None)>, <CrossValidationSplitStrategy.adaptive_peak_disease_timepoints: CrossV

# GeneLocus.BCR, TargetObsColumnEnum.disease trained on train_smaller set

## Specimen predictions on validation set

elasticnet_cv0.25,elasticnet_cv,elasticnet_sklearn_with_lambdamax,elasticnet0.25_sklearn_with_lambdamax
Per-fold scores: ROC-AUC (weighted OvO): 0.929 +/- 0.003 (in 3 folds) ROC-AUC (macro OvO): 0.935 +/- 0.004 (in 3 folds) au-PRC (weighted OvO): 0.930 +/- 0.006 (in 3 folds) au-PRC (macro OvO): 0.938 +/- 0.009 (in 3 folds) Accuracy: 0.747 +/- 0.051 (in 3 folds) MCC: 0.678 +/- 0.062 (in 3 folds) Global scores: Accuracy: 0.746 MCC: 0.677 Global classification report: precision recall f1-score support Covid19 0.81 0.81 0.81 43 HIV 0.88 0.91 0.89 64 Healthy/Background 0.80 0.74 0.77 150 Influenza 0.63 0.71 0.67 24 Lupus 0.68 0.66 0.67 65 T1D 0.58 0.66 0.61 64 accuracy 0.75 410 macro avg 0.73 0.75 0.74 410 weighted avg 0.75 0.75 0.75 410,Per-fold scores: ROC-AUC (weighted OvO): 0.929 +/- 0.003 (in 3 folds) ROC-AUC (macro OvO): 0.935 +/- 0.004 (in 3 folds) au-PRC (weighted OvO): 0.930 +/- 0.006 (in 3 folds) au-PRC (macro OvO): 0.938 +/- 0.007 (in 3 folds) Accuracy: 0.739 +/- 0.046 (in 3 folds) MCC: 0.669 +/- 0.056 (in 3 folds) Global scores: Accuracy: 0.739 MCC: 0.668 Global classification report: precision recall f1-score support Covid19 0.85 0.81 0.83 43 HIV 0.85 0.91 0.88 64 Healthy/Background 0.79 0.72 0.76 150 Influenza 0.65 0.71 0.68 24 Lupus 0.66 0.66 0.66 65 T1D 0.57 0.66 0.61 64 accuracy 0.74 410 macro avg 0.73 0.74 0.74 410 weighted avg 0.74 0.74 0.74 410,Per-fold scores: ROC-AUC (weighted OvO): 0.929 +/- 0.003 (in 3 folds) ROC-AUC (macro OvO): 0.935 +/- 0.004 (in 3 folds) au-PRC (weighted OvO): 0.930 +/- 0.005 (in 3 folds) au-PRC (macro OvO): 0.938 +/- 0.007 (in 3 folds) Accuracy: 0.739 +/- 0.046 (in 3 folds) MCC: 0.669 +/- 0.056 (in 3 folds) Global scores: Accuracy: 0.739 MCC: 0.668 Global classification report: precision recall f1-score support Covid19 0.83 0.81 0.82 43 HIV 0.85 0.91 0.88 64 Healthy/Background 0.80 0.73 0.76 150 Influenza 0.65 0.71 0.68 24 Lupus 0.66 0.65 0.65 65 T1D 0.58 0.66 0.61 64 accuracy 0.74 410 macro avg 0.73 0.74 0.73 410 weighted avg 0.74 0.74 0.74 410,Per-fold scores: ROC-AUC (weighted OvO): 0.929 +/- 0.003 (in 3 folds) ROC-AUC (macro OvO): 0.935 +/- 0.004 (in 3 folds) au-PRC (weighted OvO): 0.930 +/- 0.007 (in 3 folds) au-PRC (macro OvO): 0.938 +/- 0.009 (in 3 folds) Accuracy: 0.747 +/- 0.051 (in 3 folds) MCC: 0.678 +/- 0.062 (in 3 folds) Global scores: Accuracy: 0.746 MCC: 0.677 Global classification report: precision recall f1-score support Covid19 0.81 0.81 0.81 43 HIV 0.88 0.91 0.89 64 Healthy/Background 0.80 0.74 0.77 150 Influenza 0.63 0.71 0.67 24 Lupus 0.68 0.66 0.67 65 T1D 0.58 0.66 0.61 64 accuracy 0.75 410 macro avg 0.73 0.75 0.74 410 weighted avg 0.75 0.75 0.75 410
,,,
,,,
,,,


elasticnet0.75_sklearn_with_lambdamax,elasticnet_cv0.75,lasso_cv,lasso_sklearn_with_lambdamax
Per-fold scores: ROC-AUC (weighted OvO): 0.929 +/- 0.001 (in 3 folds) ROC-AUC (macro OvO): 0.936 +/- 0.002 (in 3 folds) au-PRC (weighted OvO): 0.931 +/- 0.004 (in 3 folds) au-PRC (macro OvO): 0.939 +/- 0.006 (in 3 folds) Accuracy: 0.739 +/- 0.049 (in 3 folds) MCC: 0.670 +/- 0.059 (in 3 folds) Global scores: Accuracy: 0.739 MCC: 0.669 Global classification report: precision recall f1-score support Covid19 0.85 0.81 0.83 43 HIV 0.87 0.92 0.89 64 Healthy/Background 0.79 0.72 0.76 150 Influenza 0.71 0.71 0.71 24 Lupus 0.67 0.65 0.66 65 T1D 0.54 0.66 0.59 64 accuracy 0.74 410 macro avg 0.74 0.74 0.74 410 weighted avg 0.75 0.74 0.74 410,Per-fold scores: ROC-AUC (weighted OvO): 0.928 +/- 0.001 (in 3 folds) ROC-AUC (macro OvO): 0.936 +/- 0.002 (in 3 folds) au-PRC (weighted OvO): 0.930 +/- 0.004 (in 3 folds) au-PRC (macro OvO): 0.938 +/- 0.006 (in 3 folds) Accuracy: 0.739 +/- 0.049 (in 3 folds) MCC: 0.670 +/- 0.059 (in 3 folds) Global scores: Accuracy: 0.739 MCC: 0.669 Global classification report: precision recall f1-score support Covid19 0.85 0.81 0.83 43 HIV 0.87 0.92 0.89 64 Healthy/Background 0.79 0.72 0.76 150 Influenza 0.71 0.71 0.71 24 Lupus 0.67 0.65 0.66 65 T1D 0.54 0.66 0.59 64 accuracy 0.74 410 macro avg 0.74 0.74 0.74 410 weighted avg 0.75 0.74 0.74 410,Per-fold scores: ROC-AUC (weighted OvO): 0.927 +/- 0.002 (in 3 folds) ROC-AUC (macro OvO): 0.934 +/- 0.001 (in 3 folds) au-PRC (weighted OvO): 0.927 +/- 0.004 (in 3 folds) au-PRC (macro OvO): 0.935 +/- 0.005 (in 3 folds) Accuracy: 0.720 +/- 0.035 (in 3 folds) MCC: 0.645 +/- 0.042 (in 3 folds) Global scores: Accuracy: 0.720 MCC: 0.645 Global classification report: precision recall f1-score support Covid19 0.81 0.81 0.81 43 HIV 0.83 0.92 0.87 64 Healthy/Background 0.79 0.70 0.74 150 Influenza 0.64 0.67 0.65 24 Lupus 0.64 0.65 0.64 65 T1D 0.53 0.59 0.56 64 accuracy 0.72 410 macro avg 0.71 0.72 0.71 410 weighted avg 0.72 0.72 0.72 410,Per-fold scores: ROC-AUC (weighted OvO): 0.927 +/- 0.002 (in 3 folds) ROC-AUC (macro OvO): 0.934 +/- 0.002 (in 3 folds) au-PRC (weighted OvO): 0.928 +/- 0.004 (in 3 folds) au-PRC (macro OvO): 0.936 +/- 0.005 (in 3 folds) Accuracy: 0.722 +/- 0.031 (in 3 folds) MCC: 0.648 +/- 0.037 (in 3 folds) Global scores: Accuracy: 0.722 MCC: 0.648 Global classification report: precision recall f1-score support Covid19 0.85 0.81 0.83 43 HIV 0.83 0.92 0.87 64 Healthy/Background 0.79 0.70 0.74 150 Influenza 0.64 0.67 0.65 24 Lupus 0.64 0.65 0.64 65 T1D 0.53 0.61 0.57 64 accuracy 0.72 410 macro avg 0.71 0.73 0.72 410 weighted avg 0.73 0.72 0.72 410
,,,
,,,
,,,


...

ridge_cv,ridge_cv.decision_thresholds_tuned,logisticregression_unregularized.decision_thresholds_tuned,logisticregression_unregularized
Per-fold scores: ROC-AUC (weighted OvO): 0.926 +/- 0.009 (in 3 folds) ROC-AUC (macro OvO): 0.928 +/- 0.011 (in 3 folds) au-PRC (weighted OvO): 0.915 +/- 0.014 (in 3 folds) au-PRC (macro OvO): 0.919 +/- 0.015 (in 3 folds) Accuracy: 0.667 +/- 0.018 (in 3 folds) MCC: 0.595 +/- 0.024 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.667 MCC: 0.595 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.54 0.59 0.56 58 HIV 0.70 0.80 0.75 98 Healthy/Background 0.88 0.60 0.72 197 Influenza 0.55 0.89 0.68 37 Lupus 0.45 0.61 0.52 64 T1D 0.67 0.67 0.67 96 accuracy 0.67 550 macro avg 0.63 0.69 0.65 550 weighted avg 0.71 0.67 0.67 550,Per-fold scores: ROC-AUC (weighted OvO): 0.926 +/- 0.009 (in 3 folds) ROC-AUC (macro OvO): 0.928 +/- 0.011 (in 3 folds) au-PRC (weighted OvO): 0.915 +/- 0.014 (in 3 folds) au-PRC (macro OvO): 0.919 +/- 0.015 (in 3 folds) Accuracy: 0.673 +/- 0.047 (in 3 folds) MCC: 0.590 +/- 0.067 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.673 MCC: 0.587 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.57 0.59 0.58 58 HIV 0.70 0.76 0.73 98 Healthy/Background 0.79 0.68 0.73 197 Influenza 0.65 0.70 0.68 37 Lupus 0.48 0.52 0.50 64 T1D 0.64 0.72 0.68 96 accuracy 0.67 550 macro avg 0.64 0.66 0.65 550 weighted avg 0.68 0.67 0.67 550,Per-fold scores: ROC-AUC (weighted OvO): 0.926 +/- 0.007 (in 3 folds) ROC-AUC (macro OvO): 0.926 +/- 0.008 (in 3 folds) au-PRC (weighted OvO): 0.919 +/- 0.013 (in 3 folds) au-PRC (macro OvO): 0.921 +/- 0.014 (in 3 folds) Accuracy: 0.705 +/- 0.021 (in 3 folds) MCC: 0.633 +/- 0.018 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.705 MCC: 0.631 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.52 0.59 0.55 58 HIV 0.70 0.80 0.74 98 Healthy/Background 0.85 0.72 0.78 197 Influenza 0.58 0.76 0.66 37 Lupus 0.64 0.58 0.61 64 T1D 0.69 0.73 0.71 96 accuracy 0.71 550 macro avg 0.66 0.69 0.67 550 weighted avg 0.72 0.71 0.71 550,Per-fold scores: ROC-AUC (weighted OvO): 0.926 +/- 0.007 (in 3 folds) ROC-AUC (macro OvO): 0.926 +/- 0.008 (in 3 folds) au-PRC (weighted OvO): 0.919 +/- 0.013 (in 3 folds) au-PRC (macro OvO): 0.921 +/- 0.014 (in 3 folds) Accuracy: 0.702 +/- 0.022 (in 3 folds) MCC: 0.623 +/- 0.033 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.702 MCC: 0.623 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.52 0.55 0.53 58 HIV 0.73 0.73 0.73 98 Healthy/Background 0.82 0.76 0.79 197 Influenza 0.59 0.78 0.67 37 Lupus 0.57 0.64 0.60 64 T1D 0.73 0.66 0.69 96 accuracy 0.70 550 macro avg 0.66 0.69 0.67 550 weighted avg 0.71 0.70 0.70 550
,,,
,,,


linearsvm_ovr,linearsvm_ovr.decision_thresholds_tuned,rf_multiclass,rf_multiclass.decision_thresholds_tuned
Per-fold scores: ROC-AUC (weighted OvO): 0.923 +/- 0.010 (in 3 folds) ROC-AUC (macro OvO): 0.924 +/- 0.012 (in 3 folds) au-PRC (weighted OvO): 0.913 +/- 0.011 (in 3 folds) au-PRC (macro OvO): 0.916 +/- 0.013 (in 3 folds) Accuracy: 0.695 +/- 0.010 (in 3 folds) MCC: 0.618 +/- 0.009 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.695 MCC: 0.618 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.60 0.55 0.58 58 HIV 0.70 0.82 0.75 98 Healthy/Background 0.85 0.71 0.77 197 Influenza 0.55 0.76 0.64 37 Lupus 0.51 0.58 0.54 64 T1D 0.68 0.69 0.68 96 accuracy 0.69 550 macro avg 0.65 0.68 0.66 550 weighted avg 0.71 0.69 0.70 550,Per-fold scores: ROC-AUC (weighted OvO): 0.923 +/- 0.010 (in 3 folds) ROC-AUC (macro OvO): 0.924 +/- 0.012 (in 3 folds) au-PRC (weighted OvO): 0.913 +/- 0.011 (in 3 folds) au-PRC (macro OvO): 0.916 +/- 0.013 (in 3 folds) Accuracy: 0.684 +/- 0.033 (in 3 folds) MCC: 0.607 +/- 0.036 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.684 MCC: 0.606 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.67 0.50 0.57 58 HIV 0.65 0.87 0.75 98 Healthy/Background 0.84 0.69 0.76 197 Influenza 0.48 0.78 0.59 37 Lupus 0.56 0.50 0.53 64 T1D 0.66 0.68 0.67 96 accuracy 0.68 550 macro avg 0.65 0.67 0.65 550 weighted avg 0.70 0.68 0.68 550,Per-fold scores: ROC-AUC (weighted OvO): 0.920 +/- 0.023 (in 3 folds) ROC-AUC (macro OvO): 0.921 +/- 0.023 (in 3 folds) au-PRC (weighted OvO): 0.908 +/- 0.029 (in 3 folds) au-PRC (macro OvO): 0.912 +/- 0.027 (in 3 folds) Accuracy: 0.692 +/- 0.063 (in 3 folds) MCC: 0.601 +/- 0.082 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.693 MCC: 0.601 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.69 0.43 0.53 58 HIV 0.74 0.77 0.75 98 Healthy/Background 0.70 0.80 0.75 197 Influenza 0.70 0.51 0.59 37 Lupus 0.62 0.48 0.54 64 T1D 0.67 0.77 0.71 96 accuracy 0.69 550 macro avg 0.69 0.63 0.65 550 weighted avg 0.69 0.69 0.68 550,Per-fold scores: ROC-AUC (weighted OvO): 0.920 +/- 0.023 (in 3 folds) ROC-AUC (macro OvO): 0.921 +/- 0.023 (in 3 folds) au-PRC (weighted OvO): 0.908 +/- 0.029 (in 3 folds) au-PRC (macro OvO): 0.912 +/- 0.027 (in 3 folds) Accuracy: 0.670 +/- 0.068 (in 3 folds) MCC: 0.588 +/- 0.072 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.671 MCC: 0.583 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.46 0.55 0.50 58 HIV 0.66 0.87 0.75 98 Healthy/Background 0.80 0.73 0.76 197 Influenza 0.77 0.46 0.58 37 Lupus 0.52 0.53 0.53 64 T1D 0.67 0.60 0.64 96 accuracy 0.67 550 macro avg 0.65 0.62 0.63 550 weighted avg 0.68 0.67 0.67 550
,,,
,,,


xgboost,xgboost.decision_thresholds_tuned,elasticnet_cv_each_disease_versus_healthy,elasticnet_cv_each_disease_versus_healthy.decision_thresholds_tuned
Per-fold scores: ROC-AUC (weighted OvO): 0.918 +/- 0.025 (in 3 folds) ROC-AUC (macro OvO): 0.919 +/- 0.024 (in 3 folds) au-PRC (weighted OvO): 0.914 +/- 0.029 (in 3 folds) au-PRC (macro OvO): 0.917 +/- 0.028 (in 3 folds) Accuracy: 0.701 +/- 0.047 (in 3 folds) MCC: 0.616 +/- 0.061 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.702 MCC: 0.616 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.62 0.52 0.57 58 HIV 0.70 0.71 0.71 98 Healthy/Background 0.75 0.78 0.76 197 Influenza 0.61 0.59 0.60 37 Lupus 0.64 0.67 0.66 64 T1D 0.72 0.71 0.71 96 accuracy 0.70 550 macro avg 0.67 0.66 0.67 550 weighted avg 0.70 0.70 0.70 550,Per-fold scores: ROC-AUC (weighted OvO): 0.918 +/- 0.025 (in 3 folds) ROC-AUC (macro OvO): 0.919 +/- 0.024 (in 3 folds) au-PRC (weighted OvO): 0.914 +/- 0.029 (in 3 folds) au-PRC (macro OvO): 0.917 +/- 0.028 (in 3 folds) Accuracy: 0.696 +/- 0.059 (in 3 folds) MCC: 0.616 +/- 0.070 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.696 MCC: 0.614 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.68 0.45 0.54 58 HIV 0.69 0.80 0.74 98 Healthy/Background 0.79 0.73 0.75 197 Influenza 0.57 0.54 0.56 37 Lupus 0.59 0.66 0.62 64 T1D 0.67 0.77 0.71 96 accuracy 0.70 550 macro avg 0.66 0.66 0.65 550 weighted avg 0.70 0.70 0.69 550,Per-fold scores: ROC-AUC (weighted OvO): 0.786 +/- 0.018 (in 3 folds) ROC-AUC (macro OvO): 0.804 +/- 0.023 (in 3 folds) au-PRC (weighted OvO): 0.822 +/- 0.023 (in 3 folds) au-PRC (macro OvO): 0.826 +/- 0.029 (in 3 folds) Accuracy: 0.430 +/- 0.047 (in 3 folds) MCC: 0.373 +/- 0.066 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.429 MCC: 0.371 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.22 0.31 0.26 58 HIV 0.59 0.86 0.70 98 Healthy/Background 0.00 0.00 0.00 197 Influenza 0.34 0.81 0.48 37 Lupus 0.27 0.36 0.31 64 T1D 0.54 0.84 0.66 96 accuracy 0.43 550 macro avg 0.33 0.53 0.40 550 weighted avg 0.28 0.43 0.33 550,Per-fold scores: ROC-AUC (weighted OvO): 0.786 +/- 0.018 (in 3 folds) ROC-AUC (macro OvO): 0.804 +/- 0.023 (in 3 folds) au-PRC (weighted OvO): 0.822 +/- 0.023 (in 3 folds) au-PRC (macro OvO): 0.826 +/- 0.029 (in 3 folds) Accuracy: 0.448 +/- 0.037 (in 3 folds) MCC: 0.407 +/- 0.049 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.447 MCC: 0.404 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.33 0.67 0.44 58 HIV 0.60 0.81 0.69 98 Healthy/Background 0.00 0.00 0.00 197 Influenza 0.30 0.86 0.44 37 Lupus 0.36 0.56 0.44 64 T1D 0.66 0.62 0.64 96 accuracy 0.45 550 macro avg 0.37 0.59 0.44 550 weighted avg 0.32 0.45 0.36 550
,,,
,,,


dummy_stratified.decision_thresholds_tuned,dummy_stratified,dummy_most_frequent.decision_thresholds_tuned,dummy_most_frequent
Per-fold scores: ROC-AUC (weighted OvO): 0.505 +/- 0.013 (in 3 folds) ROC-AUC (macro OvO): 0.502 +/- 0.014 (in 3 folds) au-PRC (weighted OvO): 0.507 +/- 0.006 (in 3 folds) au-PRC (macro OvO): 0.507 +/- 0.007 (in 3 folds) Accuracy: 0.219 +/- 0.038 (in 3 folds) MCC: 0.014 +/- 0.027 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.218 MCC: 0.014 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.15 0.19 0.17 58 HIV 0.18 0.17 0.18 98 Healthy/Background 0.39 0.35 0.37 197 Influenza 0.00 0.00 0.00 37 Lupus 0.16 0.19 0.17 64 T1D 0.13 0.12 0.13 96 accuracy 0.22 550 macro avg 0.17 0.17 0.17 550 weighted avg 0.23 0.22 0.22 550,Per-fold scores: ROC-AUC (weighted OvO): 0.505 +/- 0.013 (in 3 folds) ROC-AUC (macro OvO): 0.502 +/- 0.014 (in 3 folds) au-PRC (weighted OvO): 0.507 +/- 0.006 (in 3 folds) au-PRC (macro OvO): 0.507 +/- 0.007 (in 3 folds) Accuracy: 0.219 +/- 0.038 (in 3 folds) MCC: 0.014 +/- 0.027 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.218 MCC: 0.014 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.15 0.19 0.17 58 HIV 0.18 0.17 0.18 98 Healthy/Background 0.39 0.35 0.37 197 Influenza 0.00 0.00 0.00 37 Lupus 0.16 0.19 0.17 64 T1D 0.13 0.12 0.13 96 accuracy 0.22 550 macro avg 0.17 0.17 0.17 550 weighted avg 0.23 0.22 0.22 550,Per-fold scores: ROC-AUC (weighted OvO): 0.500 +/- 0.000 (in 3 folds) ROC-AUC (macro OvO): 0.500 +/- 0.000 (in 3 folds) au-PRC (weighted OvO): 0.500 +/- 0.000 (in 3 folds) au-PRC (macro OvO): 0.500 +/- 0.000 (in 3 folds) Accuracy: 0.358 +/- 0.003 (in 3 folds) MCC: 0.000 +/- 0.000 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.358 MCC: 0.000 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.00 0.00 0.00 58 HIV 0.00 0.00 0.00 98 Healthy/Background 0.36 1.00 0.53 197 Influenza 0.00 0.00 0.00 37 Lupus 0.00 0.00 0.00 64 T1D 0.00 0.00 0.00 96 accuracy 0.36 550 macro avg 0.06 0.17 0.09 550 weighted avg 0.13 0.36 0.19 550,Per-fold scores: ROC-AUC (weighted OvO): 0.500 +/- 0.000 (in 3 folds) ROC-AUC (macro OvO): 0.500 +/- 0.000 (in 3 folds) au-PRC (weighted OvO): 0.500 +/- 0.000 (in 3 folds) au-PRC (macro OvO): 0.500 +/- 0.000 (in 3 folds) Accuracy: 0.358 +/- 0.003 (in 3 folds) MCC: 0.000 +/- 0.000 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.358 MCC: 0.000 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.00 0.00 0.00 58 HIV 0.00 0.00 0.00 98 Healthy/Background 0.36 1.00 0.53 197 Influenza 0.00 0.00 0.00 37 Lupus 0.00 0.00 0.00 64 T1D 0.00 0.00 0.00 96 accuracy 0.36 550 macro avg 0.06 0.17 0.09 550 weighted avg 0.13 0.36 0.19 550
,,,
,,,


---